Following the first part of the workshop ["Diving into NumPy
Code"](https://www.youtube.com/watch?v=G-Iep_MnSv8), held at SciPy2013. 

(It is this kind of tutorial that starts somewhere in the middle. I used LLM to
understand this better and took notes here. Here are some things that I have learned or
was reminded of:)

In [ ]:
import numpy as np

# creating and changing a numpy dtype
d = np.dtype(np.float32).newbyteorder('>') # big endian
d

dtype('>f4')

In [ ]:
# scalar arrays are like a bridge between numpy arrays and python scalars

arr = np.ones((2,2))
arr[0,0]
# np.float64(1.0)

type(arr[0,0]) 
# I am pretty sure that is not a scalar array, but a usual numpy dtype. Has numpy
# changed this since 2013?

numpy.float64

In [ ]:
# getting the transpose is basically for free, because we only need to swap the strides:

arr = np.ones((2,2))
arr.strides
# (16, 8) 

# the first number is the bytes to pass in memory to get to the next row
# the second number is the bytes to pass in memory to get to the next column

# 8 bytes to the next column, because we have a np.int64 dtype and that means that we
# use 8 bytes altogether

arr.flags
# C_CONTIGUOUS : True
# F_CONTIGUOUS : False
# OWNDATA : True
# WRITEABLE : True
# ALIGNED : True
# WRITEBACKIFCOPY : False

arr.T.strides # `arr.T` is a view, not a copy, achieved by swapping the strides
# (8, 16)

arr.T.flags
# C_CONTIGUOUS : False
# F_CONTIGUOUS : True
# OWNDATA : False # since the Transpose is a view, it doesn't own the memory
# WRITEABLE : True
# ALIGNED : True
# WRITEBACKIFCOPY : False

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [ ]:
# broadcasting

x = np.zeros((3,5)) # interpretation: 3 rows, 5 columns
y = np.zeros(8)

x[..., np.newaxis].shape # adding a new axis to prepare broadcasting
# (3, 5, 1) # interpretation: 3 batches, 5 rows, 1 column

(x[..., np.newaxis] + y).shape
# (3, 5, 8) # interpretation: 3 batches, 5 rows, 8 columns

(3, 5, 8)

In [ ]:
# without adding new axis:
x + y

ValueError: operands could not be broadcast together with shapes (3,5) (8,) 

In broadcasting, shapes are matched from right to left. Broadcasting is possible, if
either all the dimensions are equal or are 1 or None:
```
   (5, 10)      (5, 10)     (5, 10, 1)
(3, 5, 10)      (6, 10)        (10, 5)
----------      -------     ----------
(3, 5, 10) OK   BAD         (5, 10, 5) OK
```

In [ ]:
# checking strides after adding new axes

x.strides
# (40, 8)

x[..., np.newaxis].strides
# (40, 8, 0)

(40, 8, 0)

Combining indexing and broadcasting

In [ ]:
x = np.array([[1, 2], [3, 4]])
x

array([[1, 2],
       [3, 4]])

In [ ]:
# creating indexing arrays
ix0 = np.array([0, 0, 1, 1]) # shape (4,)
# array([0, 0, 1, 1])

ix1 = np.array([[1], [0]]) # shape (2, 1)
# array([[1],
#        [0]])

In [ ]:
# indexing with broadcasting
x[ix0, ix1]

array([[2, 2, 4, 4],
       [1, 1, 3, 3]])

In [ ]:
# let's discover what happens while indexing with broadcasting:

# first the two indexing arrays are broadcast together

# this is how we can see what that gives us:
np.broadcast_arrays(ix0, ix1) # returns both arrays, but broadcast together

# seems that broadcasting has nothing to do with any element-wise math operation that
# goes on between two arrays, as I had previously thought; seems to be the step before

(array([[0, 0, 1, 1],
        [0, 0, 1, 1]]),
 array([[1, 1, 1, 1],
        [0, 0, 0, 0]]))

In [ ]:
# these are then used to index x one after the other
# the elements of the new first row are x[0,1], then x[0,1], then x[1,1] and x[1,1]
# for the second row these are x[0,0], x[0,0], x[1,0] and x[1,0]

x
# array([[1, 2],
#        [3, 4]])

x[ix0, ix1]

array([[2, 2, 4, 4],
       [1, 1, 3, 3]])

In [42]:
# a more intuitive example

a = np.array([1,2,3])
b = np.array([[1], [2]])

np.broadcast_arrays(a, b)
# (array([[1, 2, 3],
#         [1, 2, 3]]),
#  array([[1, 1, 1],
#         [2, 2, 2]]))

a+b

array([[2, 3, 4],
       [3, 4, 5]])